<a href="https://colab.research.google.com/github/herysedra/covid19-mankaiza-clone/blob/andrana/scripts/paper/simple_blocks/simple_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pymc3 as pm
import theano.tensor as tt
import theano
import datetime
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import matplotlib
import pickle


In [62]:
num_days = 5

a = tt.ones(num_days-1)
b = np.array(a)
b

array(Alloc.0, dtype=object)

In [0]:
print(tt.ones(num_days-1))

In [0]:
type(tt.ones(num_days-1))

theano.tensor.var.TensorVariable

In [0]:
tt.ones(num_days-1).shape

Shape.0

In [0]:
2*tt.ones(num_days-1)

Elemwise{mul,no_inplace}.0

In [0]:
print(2*tt.ones(num_days-1))

Elemwise{mul,no_inplace}.0


In [0]:
type(2*tt.ones(num_days-1))

theano.tensor.var.TensorVariable

In [0]:
2*tt.ones(num_days-1).shape

Elemwise{mul,no_inplace}.0

In [5]:
a = np.array([1, 2, 3, 4])
a

array([1, 2, 3, 4])

In [32]:
b = range(10)
type(a)

numpy.ndarray

In [38]:
A = np.array([[1,2,3,4],[5,6,7,8]])
print(A)
A[:,2] # returns the third columm

[[1 2 3 4]
 [5 6 7 8]]


array([3, 7])

In [41]:
k = tt.iscalar("k")
A = tt.vector("A")

# Symbolic description of the result
result, updates = theano.scan(fn=lambda prior_result, A: prior_result * A,
                              outputs_info=tt.ones_like(A),
                              non_sequences=A,
                              n_steps=k)

# We only care about A**k, but scan has provided us with A**1 through A**k.
# result.shape is equal to (A.shape)**k 
# Discard the values that we don't care about. Scan is smart enough to
# notice this and not waste memory saving them.

final_result = result[-1]



# compiled function that returns A**k
power = theano.function(inputs=[A,k], outputs=final_result, updates=updates)
bb = theano.function([A,k], outputs=result)

print(power(range(10),2))
print(power(range(10),4))
print(bb(range(10),5))


[ 0.  1.  4.  9. 16. 25. 36. 49. 64. 81.]
[0.000e+00 1.000e+00 1.600e+01 8.100e+01 2.560e+02 6.250e+02 1.296e+03
 2.401e+03 4.096e+03 6.561e+03]
[[0.0000e+00 1.0000e+00 2.0000e+00 3.0000e+00 4.0000e+00 5.0000e+00
  6.0000e+00 7.0000e+00 8.0000e+00 9.0000e+00]
 [0.0000e+00 1.0000e+00 4.0000e+00 9.0000e+00 1.6000e+01 2.5000e+01
  3.6000e+01 4.9000e+01 6.4000e+01 8.1000e+01]
 [0.0000e+00 1.0000e+00 8.0000e+00 2.7000e+01 6.4000e+01 1.2500e+02
  2.1600e+02 3.4300e+02 5.1200e+02 7.2900e+02]
 [0.0000e+00 1.0000e+00 1.6000e+01 8.1000e+01 2.5600e+02 6.2500e+02
  1.2960e+03 2.4010e+03 4.0960e+03 6.5610e+03]
 [0.0000e+00 1.0000e+00 3.2000e+01 2.4300e+02 1.0240e+03 3.1250e+03
  7.7760e+03 1.6807e+04 3.2768e+04 5.9049e+04]]


In [42]:
u = tt.vector('u')
#u = [1, 2, 3, 4]
v = tt.vector('v')
#v = [11, 12, 13, 14]

w = tt.fmatrix('w')
w = tt.concatenate([u[-4:], v])
f = theano.function([u,v], w)
f([1, 2, 3, 4],[11, 12, 13, 14])

array([ 1.,  2.,  3.,  4., 11., 12., 13., 14.])

In [45]:
u = tt.vector('u')
#u = [1, 2, 3, 4]
v = tt.vector('v')
#v = [11, 12, 13, 14]

w = tt.fmatrix('w')
w = tt.concatenate([u[-2:], v])
f = theano.function([u,v], w)
f([1, 2, 3, 4],[11, 12, 13, 14])

array([ 3.,  4., 11., 12., 13., 14.])

In [46]:
u = tt.vector('u')
#u = [1, 2, 3, 4]
v = tt.vector('v')
#v = [11, 12, 13, 14]

w = tt.fmatrix('w')
w = tt.concatenate([u, v])
f = theano.function([u,v], w)
f([1, 2, 3, 4],[11, 12, 13, 14])

array([ 1.,  2.,  3.,  4., 11., 12., 13., 14.])

In [26]:
x = tt.vector('x')

def square(x):
  return x*x

outputs, updates = theano.scan(
  fn=square,
  sequences=x,
  n_steps=x.shape[0],
)

square_op = theano.function(
  inputs=[x],
  outputs=[outputs],
)

o_val = square_op(np.array([1, 2, 3, 4, 5]))

print("output:", o_val)

# theano.scan has symbolic outputs which can be taken as outputs of another function
# and the sequence_variable of the scan as its inputs 

output: [array([ 1.,  4.,  9., 16., 25.])]


In [47]:
constant = tt.dscalar('constant')
n_iters = tt.iscalar('n_iters')
start = tt.dscalar('start')

results, _ = theano.scan(lambda result, constant: result + constant,
                         outputs_info=[start], non_sequences=[constant], n_steps=n_iters)

f = theano.function([start, constant, n_iters], results[-1])
print('f(0,2,5): {0}'.format(f(0, 2, 5)))

f(0,2,5): 10.0


In [48]:
np.ones(5)

array([1., 1., 1., 1., 1.])

In [49]:
s = (2,2)
np.ones(s)

array([[1., 1.],
       [1., 1.]])

In [57]:
x = tt.vector('x')
y = tt.vector('y')
z = tt.maximum(x,y)
f = theano.function([x,y], z)
print(f((1,1),(2,1)))
print(f((1,3),(2,1)))
print(f((1,3),(2,4)))
# Returns a variable representing the maximum element by element of a and b

[2. 1.]
[2. 3.]
[2. 4.]


In [65]:
x = tt.vector('x')
y = tt.vector('y')
z = tt.ones(3)
z[2]

Subtensor{int64}.0

In [82]:
x = tt.matrix('x')
y = tt.matrix('y')
z = tt.abs_(x - 1)
f = theano.function([x], z)
A = np.array([[1,2,3,4],[5,6,7,-8]])
f(A)
# returns the absolute value, not the norm

array([[0., 1., 2., 3.],
       [4., 5., 6., 9.]])